<a href="https://colab.research.google.com/github/JakeOh/202208_itw_java134_lab_python/blob/main/ml03_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


* Machine Learning(기계 학습)
    * 지도 학습(supervised learning): 레이블이 있는 데이터를 학습.
        * 분류(classification): 클래스를 분류.
        * 회귀(regression): 숫자를 예측.
    * 비지도 학습(unsupervised learning): 레이블이 없는 데이터를 학습.


# 문제 정의 

fish 데이터 셋에서 물고기의 무게(Weight)를 다른 변수(특성)들을 사용해서 예측.

* Perch(농어)의 무게를 길이만으로 예측.
    * Weight(관심변수, 종속변수, 타겟, 레이블) ~ Length(독립변수, 특성)
* Perch의 무게를 다른 모든 특성들을 사용해서 예측.
    * Weight ~ Length + Diagonal + Height + Width
